# Introducción práctica a LangChain

Este cuaderno resume los conceptos esenciales de LangChain con ejemplos listos para ejecutar. Consulta la documentación oficial para ampliar cada tema:
- [LangChain Overview](https://docs.langchain.com/oss/python/langchain/overview)
- [Integraciones de Proveedores](https://docs.langchain.com/oss/python/integrations/providers/overview)
- Referencia adicional: [llamacpp_chat_model_utils.ipynb](https://raw.githubusercontent.com/basaravia/document-parsing-dl-mllm/refs/heads/main/notebooks/llamacpp_chat_model_utils.ipynb)

## 1. Introducción

### ¿Qué es LangChain?
LangChain es un **kit de orquestación de IA generativa** que uniforma la interacción con modelos, memorias, herramientas y datos. Su objetivo es ayudarte a componer flujos reproducibles que combinen *prompts*, *tool calls* y *state*.

### ¿Cuándo usar LangChain, LangGraph, Deep Agents y LangSmith?
- `LangChain`: cuando necesitas componer prompts, cadenas o herramientas rápidamente con bloques reutilizables.
- `LangGraph`: cuando tu aplicación requiere grafos de control explícitos, ciclos o máquinas de estado para agentes complejos.
- `Deep Agents`: cuando priorizas agentes autónomos con razonamiento de largo aliento (planificación, reflexión y delegación intensa).
- `LangSmith`: cuando debes **depurar, observar y versionar** tus cadenas/agentes en producción con *tracing* y evaluación continua.

### Instalación y prerequisitos
1. Usa Python ≥3.10 y un entorno virtual limpio.
2. Instala `langchain-core` y los conectores que planees usar (OpenAI, AWS, Google, etc.).
3. Define tus credenciales mediante variables de entorno para no hardcodear secretos.

In [2]:
%pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]widgets]
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip uninstall -y langchain langchain-core langchain-openai langchain-aws langchain-google-vertexai langchain-community langgraph langsmith faiss-cpu


Found existing installation: langchain 1.0.4
Uninstalling langchain-1.0.4:
  Successfully uninstalled langchain-1.0.4
Found existing installation: langchain-core 1.0.3
Uninstalling langchain-core-1.0.3:
  Successfully uninstalled langchain-core-1.0.3
Found existing installation: langchain-openai 1.0.2
Uninstalling langchain-openai-1.0.2:
  Successfully uninstalled langchain-openai-1.0.2
Found existing installation: langchain-aws 1.0.0
Uninstalling langchain-aws-1.0.0:
  Successfully uninstalled langchain-aws-1.0.0
Found existing installation: langchain-google-vertexai 3.0.2
Uninstalling langchain-google-vertexai-3.0.2:
  Successfully uninstalled langchain-google-vertexai-3.0.2
Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
Found existing installation: langgraph 1.0.2
Uninstalling langgraph-1.0.2:
  Successfully uninstalled langgraph-1.0.2
Found existing installation: langsmith 0.4.13
Un

In [ ]:
%pip install -qU langchain-core langchain langchain-openai langchain-aws langchain-google-vertexai faiss-cpu langchain-community langgraph langsmith

Note: you may need to restart the kernel to use updated packages.


### Configuración rápida con proveedores administrados
Cada proveedor expone modelos de chat con ligeras variaciones. LangChain ofrece *wrappers* homogéneos para mantener la misma interfaz `invoke`.

#### AWS Bedrock (ChatBedrockConverse)
Usa el nuevo endpoint *Converse* para acceder a modelos de Anthropic, Cohere o Amazon. Requiere `AWS_REGION`, credenciales IAM y habilitar el modelo en la consola.

In [ ]:
import os
from langchain_aws import ChatBedrockConverse

bedrock_llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    region=os.getenv("AWS_REGION", "us-east-1"),
    temperature=0.2,
    max_tokens=256,
)

try:
    bedrock_reply = bedrock_llm.invoke("Resume la misión de LangChain en dos líneas.")
    print(bedrock_reply.content)
except Exception as exc:  # pragma: no cover
    print("Inicia sesión en AWS y otorga permisos a Bedrock antes de ejecutar. Error: ", exc)

#### Azure OpenAI y runtimes compatibles (llama.cpp)
Azure ofrece modelos como GPT-4o bajo contrato empresarial y el *OpenAI compatibility layer* permite reutilizar `ChatOpenAI`. El mismo cliente funciona con servidores locales que imiten la API de OpenAI (por ejemplo, `llama.cpp` con `docker run -p 12434:12434 ghcr.io/ggerganov/llama.cpp:server`).

In [3]:
import os
from langchain_openai import ChatOpenAI

# azure_llm = ChatOpenAI(
#     api_key=os.getenv("AZURE_OPENAI_API_KEY", "dummy"),
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", "https://<tu-recurso>.openai.azure.com"),
#     azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini"),
#     api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     temperature=0.2,
#     max_tokens=300,
# )

local_llm = ChatOpenAI(
    model="ai/gemma3n:latest",
    temperature=0.2,
    max_tokens=256,
    timeout=300,
    max_retries=2,
    api_key="dummy",
    base_url="http://localhost:12434/engines/llama.cpp/v1",
)

for name, client in {"llama.cpp": local_llm}.items():
    try:
        reply = client.invoke("Explica en español una linea quien eres.")
        print(f"[{name}] {reply.content}")
    except Exception as exc:  # pragma: no cover
        print(f"[{name}] Configura el endpoint antes de ejecutar. Error: {exc}")

[llama.cpp] Soy Gemma, un modelo de lenguaje grande de código abierto creado por Google DeepMind. Estoy disponible públicamente para su uso. Recibo texto e imágenes como entrada y produzco texto como salida.



#### Google Vertex AI
El SDK `langchain-google-vertexai` expone a Gemini y modelos PaLM. Necesitas autenticarte con `gcloud auth application-default login` o una cuenta de servicio con permisos.

In [ ]:
from langchain_google_vertexai import ChatVertexAI

google_llm = ChatVertexAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=256,
)

try:
    google_reply = google_llm.invoke("¿Cómo complementa Vertex AI a LangChain?")
    print(google_reply.content)
except Exception as exc:  # pragma: no cover
    print("Configura GOOGLE_APPLICATION_CREDENTIALS antes de ejecutar. Error:", exc)

### LLM `.invoke()` en acción
Usamos un `ChatPromptTemplate` para combinar mensajes del sistema y del usuario antes de llamar al modelo. Esta es la base de cualquier cadena en LangChain.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en {tema}. Responde en español, breve."),
    ("human", "{pregunta}"),
])

chain = prompt | local_llm

try:
    response = chain.invoke({
        "tema": "arquitectura de agentes",
        "pregunta": "¿Qué piezas necesitas para lanzar un agente privado?",
    })
    print(response.content)
except Exception as exc:  # pragma: no cover
    print("Levanta tu servidor llama.cpp (base_url=12434) para probar. Error:", exc)

Para lanzar un agente privado, necesitas:

1. **Un lenguaje de programación:** Python es popular, pero otros como Java o C# también sirven.
2. **Un framework de agentes:**  Como ReactiveSwift, JADE, o una implementación personalizada.
3. **Un modelo de comportamiento:** Define cómo el agente interactúa con su entorno y otros agentes.
4. **Un entorno de simulación/ejecución:**  Un lugar donde el agente pueda operar (puede ser un simulador o un sistema real).
5. **Un conjunto de datos/recursos:**  Datos para el aprendizaje, información sobre el mundo, etc.



### Streaming y *callbacks*
LangChain soporta *streaming* con `CallbackManager`. Ideal para *demos* o UX reactivas.

In [9]:
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage

streaming_llm = ChatOpenAI(
    model="ai/gemma3n:latest",
    temperature=0.2,
    max_tokens=1000,
    timeout=300,
    max_retries=2,
    api_key="dummy",
    base_url="http://localhost:12434/engines/llama.cpp/v1",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

try:
    streaming_llm.invoke(
        [HumanMessage(content="Resume qué es Docker Model Runner en un ensayo de 100 palabras.")]
    )
except Exception as exc:  # pragma: no cover
    print("Activa el endpoint con streaming antes de ejecutar. Error:", exc)


Docker Model Runner es una herramienta de código abierto diseñada para simplificar el despliegue y la ejecución de modelos de aprendizaje automático.  Actúa como un orquestador, encapsulando modelos y sus dependencias en contenedores Docker. Esto garantiza la reproducibilidad y la portabilidad, permitiendo que los modelos se ejecuten de manera consistente en diferentes entornos.

Model Runner abstrae la complejidad de la infraestructura subyacente, facilitando la gestión de recursos y el escalado.  Permite la ejecución de modelos con diferentes frameworks (TensorFlow, PyTorch, etc.) y proporciona una interfaz unificada para la monitorización y el registro de métricas.  En esencia, agiliza el ciclo de vida del modelo, desde el desarrollo hasta la producción.


## 2. Conceptos clave
A continuación se describen los bloques fundamentales del ecosistema LangChain. Cada subtema incluye un apunte práctico.

### Modelos y mensajes
Trabaja con objetos `BaseMessage` (`SystemMessage`, `HumanMessage`, `AIMessage`). Esto permite insertar metadatos y mantener historiales consistentes.

In [10]:
from langchain_core.messages import SystemMessage, HumanMessage

conversation = [
    SystemMessage(content="Eres un arquitecto de IA."),
    HumanMessage(content="Explica la diferencia entre un LLM y un chat model."),
]

try:
    reply = local_llm.invoke(conversation)
    print(reply.content)
except Exception as exc:  # pragma: no cover
    print("Arranca tu backend compatible con OpenAI para probar. Error:", exc)

¡Hola! Como arquitecto de IA, me complace explicar la diferencia entre un LLM (Large Language Model) y un Chat Model. Aunque a menudo se usan indistintamente, existen diferencias sutiles pero importantes en su arquitectura, entrenamiento y propósito.

**LLM (Large Language Model) - El Motor de Lenguaje**

Imagina un LLM como un motor de lenguaje poderoso. Es un modelo de inteligencia artificial entrenado en una cantidad masiva de datos de texto. Su objetivo principal es **predecir la siguiente palabra en una secuencia**.  Esto se logra a través de redes neuronales profundas, típicamente basadas en la arquitectura Transformer.

**Características clave de un LLM:**

*   **Entrenamiento masivo:** Se entrenan en terabytes de datos de texto provenientes de internet, libros, artículos, código, etc.
*   **Predicción de secuencias:**  Su función principal es predecir la siguiente palabra en una secuencia dada.  Esto les permite generar texto coherente y gramaticalmente correcto.
*   **Versatil

### Structured output
`with_structured_output` usa validadores (p.ej. Pydantic) para garantizar que el modelo emita un JSON válido.

In [11]:
from pydantic import BaseModel, Field

class FeatureSummary(BaseModel):
    idea: str = Field(..., description="Resumen corto")
    bullet: list[str] = Field(..., description="Beneficios clave")

structured_prompt = ChatPromptTemplate.from_messages([
    ("system", "Devuelve JSON estricto."),
    ("human", "Resume las ventajas de LangGraph."),
])

structured_chain = structured_prompt | local_llm.with_structured_output(FeatureSummary)

try:
    structured = structured_chain.invoke({})
    print(structured)
except Exception as exc:  # pragma: no cover
    print("Ejecútalo con un endpoint real para verificar el JSON. Error:", exc)

idea='LangGraph' bullet=['**Flexibilidad y Personalización:** LangGraph ofrece un marco flexible para construir flujos de trabajo de LLM personalizados. Permite a los desarrolladores definir y conectar componentes de LLM de manera modular, adaptándose a necesidades específicas.', '**Modularidad y Reutilización:**  La arquitectura modular facilita la reutilización de componentes y la creación de flujos de trabajo complejos a partir de bloques de construcción más pequeños y manejables.', '**Integración con Herramientas Externas:**  LangGraph simplifica la integración de LLMs con diversas herramientas externas, como bases de datos, APIs y otros servicios, ampliando sus capacidades.', '**Depuración y Monitoreo:** Proporciona herramientas para depurar y monitorear flujos de trabajo de LLM, facilitando la identificación y resolución de problemas.', '**Escalabilidad:**  Diseñado para escalar, LangGraph puede manejar flujos de trabajo complejos y grandes volúmenes de datos.', '**Comunidad y Ec

### Middleware (*Runnables* y tuberías)
La API de `Runnable` permite insertar pasos de logging, branching o transformaciones sin crear clases nuevas.

In [12]:
from langchain_core.runnables import RunnableLambda

def tap(inputs):
    print(f"[Middleware] Entradas: {inputs}")
    return inputs

debug_chain = tap | prompt | local_llm

try:
    debug_chain.invoke({
        "tema": "observabilidad",
        "pregunta": "¿Cómo ayuda LangSmith al monitoreo?",
    })
except Exception as exc:  # pragma: no cover
    print("Ejecuta con un LLM activo para ver el pipeline completo. Error:", exc)

[Middleware] Entradas: {'tema': 'observabilidad', 'pregunta': '¿Cómo ayuda LangSmith al monitoreo?'}


### Chains
Una cadena conecta `Prompt → LLM → Parser`. Puedes combinarlas con operadores (`|`, `.map()`, `.batch()`), o migrar a LangGraph cuando necesites mayor control.

In [13]:
from langchain_core.output_parsers import StrOutputParser

simple_chain = prompt | local_llm | StrOutputParser()

try:
    result = simple_chain.invoke({
        "tema": "memorias",
        "pregunta": "Define memoria a corto plazo en agentes."})
    print(result)
except Exception as exc:  # pragma: no cover
    print("Conecta un servidor LLM antes de ejecutar. Error:", exc)

En agentes (como IA o sistemas computacionales), la memoria a corto plazo (MCP) es un espacio de almacenamiento temporal que permite retener información relevante para tareas inmediatas.  Es como un "bloc de notas" donde se guardan datos recientes para procesarlos rápidamente, antes de que se pierdan.  Su capacidad es limitada y la información se olvida si no se refuerza o se almacena en memoria a largo plazo.



### Agentes y herramientas
Los agentes deciden qué herramienta usar: consultas SQL, navegadores, calculadoras, etc. Reutiliza `tool` o `StructuredTool` para describir entradas/salidas.

In [6]:
import langchain; print(langchain.__version__)

1.0.4


In [28]:
from langchain.agents import create_agent

@tool
def contar_palabras(texto: str) -> int:
    """Cuenta palabras en un texto."""
    return len(texto.split())

agent = create_agent(local_llm, tools=[contar_palabras])

result = agent.invoke(
    {"messages": [{"role": "user", "content": "¿Cuántas palabras hay en el siguiente texto? 'LangChain facilita la creación de aplicaciones impulsadas por LLMs.'"}]}
)

print(result["messages"],'\n')
print(result["messages"][-1].content)

[HumanMessage(content="¿Cuántas palabras hay en el siguiente texto? 'LangChain facilita la creación de aplicaciones impulsadas por LLMs.'", additional_kwargs={}, response_metadata={}, id='2824a34c-881d-4f6b-83fe-a16bf599d78f'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 264, 'total_tokens': 319, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3n:latest', 'system_fingerprint': 'b1-c22473b', 'id': 'chatcmpl-5epDJu8WQhB9d1XyO0xaLpHpgkgpoQsQ', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--c17716ad-fde3-4211-9f9a-01112262b17a-0', tool_calls=[{'name': 'contar_palabras', 'args': {'texto': 'LangChain facilita la creación de aplicaciones impulsadas por LLMs.'}, 'id': 'gqncooJKPXbyj3gsDpqt4ZHUyb37Yc8p', 'type': 'tool_call'}], usage_metadata={'input_tokens': 264, 'output_tokens': 55, 'total_tokens': 319, 'input_token_det

### Short-term memory
La memoria de corto plazo almacena los últimos turnos para mantener contexto sin saturar el prompt.

In [30]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  

agent2 = create_agent(local_llm, tools=[contar_palabras])


agent2.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='43aecbfa-4357-42c6-ab8a-62b35ab9e2f6'),
  AIMessage(content="Hi Bob! It's nice to meet you.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 247, 'total_tokens': 268, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3n:latest', 'system_fingerprint': 'b1-c22473b', 'id': 'chatcmpl-UQcqGpa2m6kVLlyEPcYrfarCIg8BNsSF', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--1d27537e-c7cc-4578-ac74-de5664ec7b83-0', usage_metadata={'input_tokens': 247, 'output_tokens': 21, 'total_tokens': 268, 'input_token_details': {}, 'output_token_details': {}})]}

In [33]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver


class CustomAgentState(AgentState):  
    user_id: str
    preferences: dict

agent2 = create_agent(
    local_llm, 
    tools=[contar_palabras],
    state_schema=CustomAgentState,  
    checkpointer=InMemorySaver(),
)

# Custom state can be passed in invoke
result = agent2.invoke(
    {
        "messages": [{"role": "user", "content": "¿Cuántas palabras hay en el siguiente texto? 'Hola mundo este es mi primer agente.'"}],
        "user_id": "user_123",  
        "preferences": {"apodo": "agente_IA"}  
    },
    {"configurable": {"thread_id": "1"}})

print(result["messages"],'\n')
print(result["messages"][-1].content)

[HumanMessage(content="¿Cuántas palabras hay en el siguiente texto? 'Hola mundo este es mi primer agente.'", additional_kwargs={}, response_metadata={}, id='bbe0344e-0e94-4c69-841a-8d42c0b363ce'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 259, 'total_tokens': 309, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3n:latest', 'system_fingerprint': 'b1-c22473b', 'id': 'chatcmpl-lT00Lmc6yCfZMx84C6ACcFk5hrrNtsG1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d64233d3-653c-42cb-93d6-dd87815d7ce3-0', tool_calls=[{'name': 'contar_palabras', 'args': {'texto': 'Hola mundo este es mi primer agente.'}, 'id': '4vLUj6sMULAYcpnEK7vomfqfSukorcaD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 50, 'total_tokens': 309, 'input_token_details': {}, 'output_token_details': {}}), ToolMessage(content='

In [39]:
from langchain.agents import create_agent, AgentState
from langchain.tools import tool, ToolRuntime


class CustomState(AgentState):
    user_id: str

@tool
def get_user_info(
    runtime: ToolRuntime
) -> str:
    """Look up user info."""
    user_id = runtime.state["user_id"]
    return "El usuario es John Smith" if user_id == "user_123" else "Unknown user"

agent3 = create_agent(
    local_llm, 
    tools=[get_user_info],
    state_schema=CustomState,
)

result = agent3.invoke({
    "messages": "Dame información del usuario",
    "user_id": "user_123"
})

print(result["messages"],'\n')
print(result["messages"][-1].content)
# > User is John Smith.

[HumanMessage(content='Dame información del usuario', additional_kwargs={}, response_metadata={}, id='3c81e4d7-558d-4189-8abf-4e85fa2df526'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 214, 'total_tokens': 240, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3n:latest', 'system_fingerprint': 'b1-c22473b', 'id': 'chatcmpl-Qdfj43oLZRjEStlM2yvwtJOn5EUwkSnE', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--22aa837b-4532-432a-8cf7-a401f2139ff8-0', tool_calls=[{'name': 'get_user_info', 'args': {}, 'id': 'EJJf9XOO9Ny5D8wnERbachszRltzuJBh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 214, 'output_tokens': 26, 'total_tokens': 240, 'input_token_details': {}, 'output_token_details': {}}), ToolMessage(content='El usuario es John Smith', name='get_user_info', id='744bca82-70af-41df-b50e-af8d4c17d89d', tool_call_id

## 3. Conceptos de integraciones
LangChain trae adaptadores para fuentes de datos, *embeddings* y almacenes vectoriales. Aquí un vistazo rápido a los más usados.

### ChatModels
Puedes mezclar múltiples modelos y seleccionar en tiempo de ejecución según costo, latencia o dominio.

In [ ]:
chat_router = {
    "fast": ChatOpenAI(model="ai/qwen3", base_url="http://localhost:12434/engines/llama.cpp/v1", api_key="dummy"),
    "reliable": azure_llm,
}

selection = "fast"

try:
    answer = chat_router[selection].invoke("Dame un tip para evaluar prompts.")
    print(answer.content)
except Exception as exc:  # pragma: no cover
    print("Activa al menos un backend para rutear chats. Error:", exc)

### Text splitters
`RecursiveCharacterTextSplitter` equilibra longitud y solapamiento para RAG.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

raw_text = """LangChain facilita pipelines de IA.
Permite integrar memorias, herramientas y agentes."""

splitter = RecursiveCharacterTextSplitter(chunk_size=40, chunk_overlap=10)
chunks = splitter.split_text(raw_text)
print(chunks)

### Embedding models
Puedes usar `HuggingFaceEmbeddings`, `OpenAIEmbeddings`, `BedrockEmbeddings`, etc.

In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="ai/mxbai-embed-large",
    api_key="dummy",  
    base_url="http://localhost:12434/engines/llama.cpp/v1"
)

vector = embeddings.embed_query("some text to embed")
print(vector[0:5])
print(f"Dimensión del embedding: {len(vector)}")

[0.014573926106095314, 0.03432486206293106, -0.009266335517168045, 0.022159187123179436, -0.028040019795298576]
Dimensión del embedding: 1024


### Unstructured parser
`langchain-unstructured` delega la extracción de texto/elementos usando estrategias como `hi_res` para PDFs complejos.

In [ ]:
from langchain_unstructured import UnstructuredLoader

file_path = "data/docs/xxxxx.pdf"

loader_local = UnstructuredLoader(
    file_path=file_path,
    strategy="hi_res",
    languages=["spa"],
)

try:
    documents = loader_local.load()
    print(f"Se extrajeron {len(documents)} fragmentos.")
except FileNotFoundError:
    print("Coloca tu PDF en data/docs/xxxxx.pdf o actualiza la ruta.")

### Document loaders
Los `DocumentLoader` normalizan fuentes heterogéneas: archivos, sitios web, bases de datos.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader

# PDF
pdf_loader = PyPDFLoader("data/docs/ejemplo.pdf")

# Web
web_loader = WebBaseLoader("https://docs.langchain.com/oss/python/langchain/overview")

print("Loaders listos. Ejecuta load() cuando quieras materializar los documentos.")

### Vector stores (FAISS local)
FAISS es un índice vectorial rápido para búsquedas semánticas en local.

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# Assuming you have your documents and embeddings initialized
embeddings = OpenAIEmbeddings(
    model="ai/granite-embedding-multilingual:latest",
    api_key="dummy",  
    base_url="http://localhost:12434/engines/llama.cpp/v1"
)

print(f'Dimension de embeddings {len(embeddings.embed_query("hello world"))}')


Dimension de embeddings 768


https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html

In [3]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document

index = faiss.IndexFlatL2(768)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore= InMemoryDocstore(),
    index_to_docstore_id={}
)

vector_store.save_local("faiss_index")


In [97]:
# Create documents with valid text content
document_1 = Document(page_content="LangChain es una herramienta de orquestación", metadata={"tipo": "descripcion"})
document_2 = Document(page_content="Los agentes pueden usar múltiples herramientas", metadata={"tipo": "funcionalidad"})
document_3 = Document(page_content="FAISS permite búsquedas vectoriales eficientes", metadata={"tipo": "componente"})

documents = [document_1, document_2, document_3]
ids = ["doc1", "doc2", "doc3"]

try:
	vector_store.add_documents(documents=documents, ids=ids)
	print("Documents added successfully")
except Exception as e:
	print(f"Error adding documents: {e}")

Documents added successfully


In [4]:
from unidecode import unidecode

def clean(text):
    import re
    text = unidecode(text)  # Convierte tildes a ASCII
    return re.sub(r'[^a-zA-Z0-9 ]', '', text)

# Aplica clean al query
query = "¿Quién controla el flujo de un agente complejo?"

results = vector_store.similarity_search(query=query, k=1)


: 

In [ ]:
new_vector_store = FAISS.load_local(
"faiss_index", embeddings, allow_dangerous_deserialization=True
)

## 4. Próximos pasos
1. Conecta LangSmith para trazar y depurar tus cadenas en producción.
2. Lleva tus agentes a LangGraph cuando necesiten ciclos o ramificaciones complejas.
3. Experimenta con `RunnableWithRetry`, `ToolNode` y `StateGraph` para robustecer tus agentes.